# Building Fast Queries on a CSV file
The aim of this project is to build faster queries for analyzing csv data

## Reading the inventory
I will use csv module to read the dataset

In [1]:
import csv
import chardet

### Check the file encoding, assuming we do not know much about the data
# with open('laptops.csv', mode='rb') as file:
#     encoded_file = file.read()
#     detected_encoding = chardet.detect(encoded_file)  #This gives ISO-8859-1
# print(detected_encoding)

with open("laptops.csv") as file:
    inventory = list(csv.reader(file))
    header = inventory[0]
    rows = inventory[1:]
    print(header,'\n')
    print(rows[:5])

['', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros'] 

[['1', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339.69'], ['2', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898.94'], ['3', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['4', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537.45'], ['5', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD', 'Intel Ir

Here are some queries that we will want to answer:
1. Given a laptop id, find the corresponding data
2. Given an amount of money, find whether there are two laptops whose total price is that given amount
3. Identify all laptops whose price falls within a given budget

## Creating Inventory Class
Now I will create a class that represents inventory that contains different types of laptops with different properties. The methods in the class will implement the queries that we want to answer about the inventory. The data will be preprocessed to make the queries run faster.

In [2]:
# from csv import reader
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
            
inventory = Inventory('laptops.csv')
header = inventory.header
print(header,'\n')
print(len(inventory.rows))

['', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros'] 

1303


## Finding a laptop from ID
Now I will implement a way to look up a laptop from a given identifier. This way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

In [3]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            reader = csv.reader(f)
            rows = list(reader)
            
        self.header = rows[0]
        self.rows = rows[1:]
        
        for row in self.rows:
            row[-1] = int(float(row[-1]))
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
            
inventory = Inventory('laptops.csv')
inventory1 = inventory.get_laptop_from_id('15')   #The csv file does not include IDs
print(inventory1)

['15', 'Apple', 'MacBook 12"', 'Ultrabook', '12', 'IPS Panel Retina Display 2304x1440', 'Intel Core M m3 1.2GHz', '8GB', '256GB SSD', 'Intel HD Graphics 615', 'macOS', '0.92kg', 1262]


## Improving the ID lookups algorithm
In the previous cell, the algorithm had a time complexity of O(R) where R is thenumber of rows. Now I will use a more efficient way to process the data using a dictionary/set, this was we will have a constant time complexity.

In [4]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            reader = csv.reader(f)
            rows = list(reader)
            
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
        
        self.if_to_row = {}
        for row in self.rows:
            self.if_to_row[row[0]] = row
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        for row in self.if_to_row:
            return self.if_to_row[laptop_id]
        return None
    
inventory = Inventory('laptops.csv')
inventory2 = inventory.get_laptop_from_id_fast('15')
print(inventory2)

['15', 'Apple', 'MacBook 12"', 'Ultrabook', '12', 'IPS Panel Retina Display 2304x1440', 'Intel Core M m3 1.2GHz', '8GB', '256GB SSD', 'Intel HD Graphics 615', 'macOS', '0.92kg', 1262]


## Comparing the performance of both algorithms
The `get_laptop_from_id()` method has time complexity O(R), R is the number of rows. The new method has time complexity of O(1) meaning it is faster, but the trade-off is with the storage size, since it has to create a dictionary first. Now I will compare the performance of both methods.

In [5]:
## Now we will compare both methods
import time
import random

ids = [str(random.randint(1000000,9999999)) for i in range(10001)]

inventory = Inventory('laptops.csv')
total_time_no_dict = 0
for i in ids:
    start = time.time()
    inventory.get_laptop_from_id(i)
    end = time.time()
    total_time_no_dict += end - start
    
total_time_dict = 0
for i in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(i)
    end = time.time()
    total_time_dict += end - start
    
print(total_time_no_dict,'\n')
print(total_time_dict)

KeyError: '2281309'

## Looking up two laptops under promotion
In a situation where a customer has a gift card and wants to purchase two laptops, the customer needs to know if they can spend the entire amount in the gift card to purchase laptops. With this next function, given a sollar amount, it can check whether it is possible to spend precisely that amount by purchasign up to two laptops.

In [8]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            reader = csv.reader(f)
            rows = list(reader)
            
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
        
        self.if_to_row = {}
        for row in self.rows:
            self.if_to_row[row[0]] = row
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        for row in self.if_to_row:
            return self.if_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, euros):
        for row in self.rows:
            if row[-1] == euros:
                return True
        for i in self.rows:
            for j in self.rows:
                if i[-1] + j[-1] == euros:
                    return True
        return False
    
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))
            

True
False


This shows that a laptop of price `1,000 Euros` is in our inventory but a laptop of price `442 Euros` is not in our inventory

## Making the lookup for two laptops under promotion run faster

In [21]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            reader = csv.reader(f)
            rows = list(reader)
            
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
        
        self.if_to_row = {}
        for row in self.rows:
            self.if_to_row[row[0]] = row
            
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        for row in self.if_to_row:
            return self.if_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, euros):
        for row in self.rows:
            if row[-1] == euros:
                return True
        for i in self.rows:
            for j in self.rows:
                if i[-1] + j[-1] == euros:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, euros):
        if euros in self.prices:
            return True
        for price in self.prices:
            if euros - price in self.prices:
                return True
        return False
    
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


## Comparing the performance of both promotion lookup algorithms

In [23]:
prices = [random.randint(100,500) for _ in range(100)]
inventory = Inventory('laptops.csv')

total_time_no_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end-start
    
total_time_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end-start
    
print(total_time_no_set)
print(total_time_set)

8.992997884750366
0.003989219665527344


As we can see in the program above, the time it takes for the function without the set takes over `(8.99/0.0039 = 2,300)` times more to run compared to the function with the set

## Finding Laptops Within a Budget
Here, binary search will be used to make the process faster

In [34]:
def row_price(row):
    return row[-1]

class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            reader = csv.reader(f)
            rows = list(reader)
            
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
        
        self.if_to_row = {}
        for row in self.rows:
            self.if_to_row[row[0]] = row
            
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key=row_price)    
        
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        for row in self.if_to_row:
            return self.if_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, euros):
        for row in self.rows:
            if row[-1] == euros:
                return True
        for i in self.rows:
            for j in self.rows:
                if i[-1] + j[-1] == euros:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, euros):
        if euros in self.prices:
            return True
        for price in self.prices:
            if euros - price in self.prices:
                return True
        return False
    
    def find_laptop_with_price(self, target_price):
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        while range_start < range_end:
            range_middle = (range_end + range_Start) // 2
            price = self.rows_by_price[range_middle][-1]
            if price == target_price:
                return range_middle
            elif value < target_value:
                range_Start = range_middle + 1
            else:
                range_end = range_middle - 1
        price = self.rows_by_price[range_start][-1]
        if price != target_price:
            return -1
        return range_start
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                return range_middle
            else:
                range_start = range_middle + 1
        price = self.rows_by_price[range_start][-1]
        if price <= target_price:
            return -1
        return range_start    
    
inventory = Inventory('laptops.csv')
print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

977
-1
